In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import csv


# For preprocessing and modeling
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE

# For Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from tensorflow.keras.regularizers import l1_l2

from lime import lime_tabular

pd.set_option('display.max_columns', None)

## Data Pre-Processing

In [15]:
# a simple function useful to make some cells idempotent
def get_original_training_data(light=True):
    train = pd.read_csv("fraudTrain.csv").drop(['Unnamed: 0'], axis=1,)
    test = pd.read_csv("fraudTest.csv").drop(['Unnamed: 0'], axis=1,)
    
    if light:
        start = 173332
        size = 400000
        end_train = start + int(size*0.85)
        end_test = end_train +  int(size*0.15)
        
        train_set = train[start: end_train]
        test_set =  train[end_train:end_test]
        
        print("train shape ", train_set.shape)
        print("test shape ", test_set.shape)
        
        return train_set, test_set
        
    
    return train, test


def create_fraud_prob_by_amt(df):

    # Step 1: Define the custom bins for the 'amt' column, with dynamic upper bound
    max_amt = df['amt'].max()  # Get the maximum amount available in the 'amt' column
    bin_edges = [0, 700, 900, 1000, 1200, max_amt]  # Use max_amt as the upper bound for the last bin
    bin_labels = ['0-700', '700-900', '900-1000', '1000-1200', f'1200-{max_amt}']

    # Step 2: Create a new feature 'amt_range' that assigns each 'amt' to the defined bins
    df['amt_range'] = pd.cut(df['amt'], bins=bin_edges, labels=bin_labels, right=False)

    # Step 3: Add 'NaN' as a valid category to the 'amt_range' column
    df['amt_range'] = df['amt_range'].cat.add_categories('NaN')

    # Handle NaN values in amt_range
    df['amt_range'] = df['amt_range'].fillna('NaN')  # Assign 'NaN' to missing values

    # Step 3: Sort data by 'trans_date_trans_time' to ensure the time order
    df = df.sort_values(by=['trans_date_trans_time'])

    # Step 4: Initialize cumulative fraud statistics for each amt_range
    df['fraud_prob_by_amt'] = 0.0  # Initialize fraud probability

    # Step 5: Calculate time-dependent fraud probability for each transaction based on amt_range
    cumulative_fraud = {label: 0 for label in bin_labels}  # Store cumulative fraud count for each amt_range
    cumulative_total = {label: 0 for label in bin_labels}  # Store cumulative total count for each amt_range

    # Use tqdm to add a progress bar
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Calculating fraud probabilities", unit="row"):
        # Get the amt_range for the current transaction
        amt_range = row['amt_range']

        # If amt_range is 'NaN', you can choose to handle it differently (e.g., assign a default probability)
        if amt_range == 'NaN':
            df.at[idx, 'fraud_prob_by_amt'] = df['is_fraud'].mean()  # or set a custom value
        else:
            # Calculate the cumulative fraud probability for the current amt_range
            if cumulative_total[amt_range] > 0:
                df.at[idx, 'fraud_prob_by_amt'] = cumulative_fraud[amt_range] / cumulative_total[amt_range]
            else:
                # If no previous transactions in the range, use the global fraud rate
                df.at[idx, 'fraud_prob_by_amt'] = df['is_fraud'].mean()

            # Update cumulative counts for amt_range
            cumulative_total[amt_range] += 1
            cumulative_fraud[amt_range] += row['is_fraud']

    # Step 6: Drop the support column 'amt_range' after the 'fraud_prob_by_amt' feature has been created
    df.drop(columns=['amt_range'], inplace=True)
    
    # Now, df contains the 'fraud_prob_by_amt' feature, which is time-dependent
    return df


def create_category_fraud_probability(data):

    # Step 1: Ensure the data is sorted by time
    # Assuming the data['trans_date_trans_time'] is already a datetime column
    data = data.sort_values(by=['category', 'trans_date_trans_time'])

    data['category_fraud_probability'] = 0.0

    # Step 3: Calculate cumulative stats for each category
    category_groups = data.groupby('category')

    # Iterate through each category group
    for category, group in tqdm(category_groups, desc="Processing categories", total=len(category_groups)):
        # Initialize cumulative counts
        total_count = 0
        fraud_count = 0

        # Iterate through each transaction in the specific category group
        for idx, row in group.iterrows():
            # Assign cumulative probability for the current transaction
            if total_count > 0:
                data.at[idx, 'category_fraud_probability'] = fraud_count / total_count
            else:
                # Default to global fraud rate for the first transaction in the category
                data.at[idx, 'category_fraud_probability'] = data['is_fraud'].mean()

            # Update cumulative counts
            total_count += 1
            fraud_count += row['is_fraud']

    # The feature 'category_fraud_probability' is now time-dependent and ready to use
    return data



def create_merchant_fraud_probability(data):

    # Step 1: Ensure the data is sorted by time
    # data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time']) --> already done my dear, already done
    data = data.sort_values(by=['merchant', 'trans_date_trans_time'])

    # Step 2: Initialize cumulative fraud statistics
    data['merchant_fraud_probability'] = 0.0

    # Step 3: Calculate cumulative stats for each merchant
    merchant_groups = data.groupby('merchant')

    ## Iterates through each merchant group
    for merchant, group in tqdm(merchant_groups, desc="Processing merchants", total=len(merchant_groups)):
        # Initialize cumulative counts
        total_count = 0
        fraud_count = 0

        ## Iteartes through each transaction of a specific mechant group
        for idx, row in group.iterrows():
            # Assign cumulative probability for the current transaction
            if total_count > 0:
                data.at[idx, 'merchant_fraud_probability'] = fraud_count / total_count
            else:
                # Default to global fraud rate for the first transaction
                ## global fraud rate provides a reasonable prior estimate for the likelihood of fraud 
                ### based on the overall (training) dataset
                ### This can make the feature more consistent across merchants, 
                ### especially for new merchants with very few transactions.
                ### INSTEAD:
                ### If you set the first fraud probability to 0, 
                ### you’re essentially assuming that there’s no chance of fraud for the first transaction, 
                ### which could bias the model.
                data.at[idx, 'merchant_fraud_probability'] = data['is_fraud'].mean()

            # Update cumulative counts
            total_count += 1
            fraud_count += row['is_fraud']

    # The feature 'merchant_fraud_probability' is now time-dependent and ready to use
    return data


def create_hourly_fraud_probability(data):
    # Step 1: Ensure the data is sorted by time
    data = data.sort_values(by=['hour', 'trans_date_trans_time'])

    # Step 2: Initialize cumulative fraud statistics
    data['hourly_fraud_probability'] = 0.0

    # Step 3: Calculate cumulative stats for each hour group
    hour_groups = data.groupby('hour')

    ## Iterate through each hour group
    for hour, group in tqdm(hour_groups, desc="Processing hours", total=len(hour_groups)):
        # Initialize cumulative counts
        total_count = 0
        fraud_count = 0

        ## Iterate through each transaction of a specific hour group
        for idx, row in group.iterrows():
            # Assign cumulative probability for the current transaction
            if total_count > 0:
                data.at[idx, 'hourly_fraud_probability'] = fraud_count / total_count
            else:
                # Default to global fraud rate for the first transaction in each group
                data.at[idx, 'hourly_fraud_probability'] = data['is_fraud'].mean()

            # Update cumulative counts
            total_count += 1
            fraud_count += row['is_fraud']

    # The feature 'hourly_fraud_probability' is now time-dependent and ready to use
    return data



# Distance between transaction and merchant locations
def haversine(lat1, lon1, lat2, lon2):
    # Radius of Earth in kilometers
    R = 6371
    # Convert degrees to radians
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    return 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1 - a))


def data_processing_updated(complete_df, historical=False):
    

    # Distance between transaction and merchant locations
    def haversine(lat1, lon1, lat2, lon2):
        # Radius of Earth in kilometers
        R = 6371
        # Convert degrees to radians
        phi1, phi2 = np.radians(lat1), np.radians(lat2)
        dphi = np.radians(lat2 - lat1)
        dlambda = np.radians(lon2 - lon1)
        a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
        return 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    def not_aggreageted_metric(data):
        # Convert datetime column
        data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])
        data['hour'] = data['trans_date_trans_time'].dt.hour
        data['day'] = data['trans_date_trans_time'].dt.day
        data['month'] = data['trans_date_trans_time'].dt.month
        data['year'] = data['trans_date_trans_time'].dt.year
        data['day_of_week'] = data['trans_date_trans_time'].dt.dayofweek

        # Calculate user age
        data['dob'] = pd.to_datetime(data['dob'])
        data['age'] = (data['trans_date_trans_time'] - data['dob']).dt.days // 365
    
        data['distance'] = haversine(data['lat'], data['long'], data['merch_lat'], data['merch_long'])

        #Task 1: Generate number of transactions per card in specific time windows
        data['trans_date'] = pd.to_datetime(data['trans_date_trans_time']).dt.date
        data['trans_hour'] = pd.to_datetime(data['trans_date_trans_time']).dt.hour
        data['trans_month'] = pd.to_datetime(data['trans_date_trans_time']).dt.to_period('M')
        data['trans_year'] = pd.to_datetime(data['trans_date_trans_time']).dt.year
        
        return data
    
    def compute_aggregated_metrics(data):
        
        data = create_category_fraud_probability(data)
        data = create_merchant_fraud_probability(data)
        data = create_fraud_prob_by_amt(data)
        data = create_hourly_fraud_probability(data)
        
        return data
    
    
    def remove_useless_columns(data):   

        # Task 5: Drop specified columns
        columns_to_drop = ['trans_date_trans_time', 'first', 'last', 'street', 'city', 'state', 
                           'lat', 'long', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long',
                          'trans_date', 'trans_hour', 'trans_month', 'trans_year', 'cc_num', 'job']

        return data.drop(columns=columns_to_drop)
    ############################################################################################
    
    # Ensure the transaction time is in datetime format
    complete_df['trans_date_trans_time'] = pd.to_datetime(complete_df['trans_date_trans_time'])

    # Sort the dataset by transaction time
    complete_df = complete_df.sort_values('trans_date_trans_time').reset_index(drop=True)
    
    
    # Creating new not aggregated metrics
    complete_df = not_aggreageted_metric(complete_df)
    
    
    if historical:
        # computing and transfer aggregated features
        complete_df = compute_aggregated_metrics(complete_df)
    
    
    # Sort the dataset by transaction time ---> VERY IMPORTANT!! Avoid reindexing int the compute_aggregated_metrics
    complete_df = complete_df.sort_values('trans_date_trans_time').reset_index(drop=True)
    
    # dropping useless columns
    complete_df = remove_useless_columns(complete_df)
    


    # Define time-based split ratios
    train_split = 0.70  # 60% for training
    val_split = 0.15    # 20% for validation
    test_split = 0.15   # 20% for testing

    # Split the dataset into training, validation, and test sets based on time
    train_size = int(len(complete_df) * 0.7)
    val_size = int(len(complete_df) * 0.15)

    train_data = complete_df[:train_size]
    val_data = complete_df[train_size:train_size+val_size]
    test_data = complete_df[train_size+val_size:]

    # Print the number of rows in each split
    print(f"Training set: {len(train_data)} rows")
    print(f"Validation set: {len(val_data)} rows")
    print(f"Test set: {len(test_data)} rows")
    
    # Calculate the frequency of fraud for each dataset
    def calculate_fraud_frequency(df):
        total_transactions = len(df)
        total_fraud = df['is_fraud'].sum()
        return (total_fraud / total_transactions) * 100

    fraud_freq_train = calculate_fraud_frequency(train_data)
    fraud_freq_val = calculate_fraud_frequency(val_data)
    fraud_freq_test = calculate_fraud_frequency(test_data)
    
    # Print the fraud frequencies for each dataset
    print(f"Fraud Frequency in Training Data: {fraud_freq_train:.2f}%")
    print(f"Fraud Frequency in Validation Data: {fraud_freq_val:.2f}%")
    print(f"Fraud Frequency in Test Data: {fraud_freq_test:.2f}%")


    return train_data, val_data, test_data
    
    


def data_processing(data):
    # Convert datetime column
    data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])
    data['hour'] = data['trans_date_trans_time'].dt.hour
    data['day'] = data['trans_date_trans_time'].dt.day
    data['month'] = data['trans_date_trans_time'].dt.month
    data['year'] = data['trans_date_trans_time'].dt.year
    data['day_of_week'] = data['trans_date_trans_time'].dt.dayofweek

    # Calculate user age
    data['dob'] = pd.to_datetime(data['dob'])
    data['age'] = (data['trans_date_trans_time'] - data['dob']).dt.days // 365

    # Distance between transaction and merchant locations
    def haversine(lat1, lon1, lat2, lon2):
        # Radius of Earth in kilometers
        R = 6371
        # Convert degrees to radians
        phi1, phi2 = np.radians(lat1), np.radians(lat2)
        dphi = np.radians(lat2 - lat1)
        dlambda = np.radians(lon2 - lon1)
        a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
        return 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    data['distance'] = haversine(data['lat'], data['long'], data['merch_lat'], data['merch_long'])

    #Task 1: Generate number of transactions per card in specific time windows
    data['trans_date'] = pd.to_datetime(data['trans_date_trans_time']).dt.date
    data['trans_hour'] = pd.to_datetime(data['trans_date_trans_time']).dt.hour
    data['trans_month'] = pd.to_datetime(data['trans_date_trans_time']).dt.to_period('M')
    data['trans_year'] = pd.to_datetime(data['trans_date_trans_time']).dt.year

    # Aggregate transactions per hour, day, month, and year
    data['transactions_per_hour'] = data.groupby(['cc_num', 'trans_date', 'trans_hour'])['cc_num'].transform('count')
    data['transactions_per_day'] = data.groupby(['cc_num', 'trans_date'])['cc_num'].transform('count')
    data['transactions_per_month'] = data.groupby(['cc_num', 'trans_month'])['cc_num'].transform('count')
    data['transactions_per_year'] = data.groupby(['cc_num', 'trans_year'])['cc_num'].transform('count')
    data['avg_amt_cc'] = data.groupby('cc_num')['amt'].transform('mean')

    # Task 2: Aggregate statistics for merchant
    # Average transaction amount per merchant
    data['avg_amt_per_merchant'] = data.groupby('merchant')['amt'].transform('mean')

    # Fraud ratio per merchant
    data['fraud_ratio_per_merchant'] = data.groupby('merchant')['is_fraud'].transform('mean')

    # Task 3: Replace category with fraud likelihood per category
    data['fraud_likelihood_per_category'] = data.groupby('category')['is_fraud'].transform('mean')
    
    # 1. Compute ratio_1: Number of fraud related to a bank / Number of total fraud
    # First, create a fraud-related bank count and total fraud count
    fraud_by_bank = data[data['is_fraud'] == 1].groupby('merchant').size()
    total_fraud = len(data[data['is_fraud'] == 1])

    # Merge fraud count by bank with the original dataframe
    data['fraud_count_bank'] = data['merchant'].map(fraud_by_bank)
    data['ratio_1'] = (data['fraud_count_bank'] / total_fraud).fillna(0)

    # 2. Compute ratio_2: Number of fraud related to a bank / Number of transactions of the bank
    # Count total transactions for each bank
    transactions_by_bank = data.groupby('merchant').size()

    # Merge transaction count by bank with the original dataframe
    data['total_transactions_bank'] = data['merchant'].map(transactions_by_bank)
    data['ratio_2'] = (data['fraud_count_bank'] / data['total_transactions_bank']).fillna(0)

    # 3. Compute total_amt_cc: Total amount spent by the same credit card
    total_amt_cc = data.groupby('cc_num')['amt'].sum()

    # Merge total amount by cc_num with the original dataframe
    data['total_amt_cc'] = data['cc_num'].map(total_amt_cc)

    # 4. Compute number_trans_cc: Total number of transactions by the same credit card
    number_trans_cc = data.groupby('cc_num').size()

    # Merge transaction count by cc_num with the original dataframe
    data['number_trans_cc'] = data['cc_num'].map(number_trans_cc)

    # Task 5: Drop specified columns
    columns_to_drop = ['trans_date_trans_time', 'first', 'last', 'street', 'city', 'state', 
                       'lat', 'long', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long',
                      'trans_date', 'trans_hour', 'trans_month', 'trans_year', 'cc_num', 'fraud_count_bank', 
                       'total_transactions_bank' ]

    data = data.drop(columns=columns_to_drop)

    return data



from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

def encode_and_scale(train_data, validation_data, test_data, historical=False):
    
    print("removing jobs missing in train dataset but existing in test dataset")
    # removing jobs missing in train dataset but existing in test dataset
    #missing_jobs = ['Software engineer', 'Operational investment banker', 'Engineer, water']

    #for job in missing_jobs:
    #    test_data = test_data[test_data['job'] != job]
    #    validation_data = validation_data[validation_data['job'] != job]  # Same for validation dataset

    print("Encode gender")
    # Encode gender
    train_data['gender'] = train_data['gender'].apply(lambda x: 1 if x == 'M' else 0)
    validation_data['gender'] = validation_data['gender'].apply(lambda x: 1 if x == 'M' else 0)  # Apply to validation
    test_data['gender'] = test_data['gender'].apply(lambda x: 1 if x == 'M' else 0)

    categorical_cols = ['merchant', 'category']
    label_encoders = {}
    
    print("Label Encoding")
    for col in categorical_cols:
        print(col)
        # Create a LabelEncoder instance
        le = LabelEncoder()
        
        print("Fit on training data and get classes")
        # Fit on training data and get classes
        train_classes = train_data[col].unique()
        le.fit(train_classes)
        
        print("Transform train_data")
        # Transform train_data
        train_data[col] = le.transform(train_data[col])
        
        print("Transform validation_data")
        # Transform validation_data
        validation_data[col] = le.transform(validation_data[col])
        
        print("Transform test_data")
        # Transform test_data
        test_data[col] = le.transform(test_data[col])

        # Save the LabelEncoder for future use
        label_encoders[col] = le
    
    print("Scale continuous variables")
    # Scale continuous variables
    if historical:
        continuous_cols = ['amt', 'city_pop', 'hour', 'day', 'month', 'year', 'day_of_week', 
                   'age', 'distance', 'category_fraud_probability', 'merchant_fraud_probability', 
                    'fraud_prob_by_amt', 'hourly_fraud_probability']
    else:
        continuous_cols = ['amt', 'city_pop', 'hour', 'day', 'month', 'year', 'day_of_week', 
                   'age', 'distance']

    scaler = StandardScaler()
    print("fit_transform train_data")
    train_data[continuous_cols] = scaler.fit_transform(train_data[continuous_cols])
    print("transform validation_data")
    validation_data[continuous_cols] = scaler.transform(validation_data[continuous_cols])  # Scale validation data
    print("transform test_data")
    test_data[continuous_cols] = scaler.transform(test_data[continuous_cols])

    return train_data, validation_data, test_data


## Model Building

In [1]:
import tensorflow as tf
from tensorflow.keras import backend as K

# Custom Focal Loss function
def focal_loss(alpha=[0.25, 0.75], gamma=2.0):
    print("Custom Focal Loss Params")
    print("alpha: ", alpha)
    print("gamma: ", gamma)
    """
    Focal Loss for binary classification with class-specific alpha.
    
    Parameters:
    - alpha: List with two values: [alpha for negative class (0), alpha for positive class (1)].
    - gamma: Focusing parameter.
    
    Returns:
    - Loss function to be used in model.compile().
    """
    def loss_fn(y_true, y_pred):
        # Cast y_true to float32 to match y_pred
        ### Ensures y_true is in float32 format for correct tensor operation
        y_true = tf.cast(y_true, tf.float32)
        
        # Clip predictions to avoid log(0)
        ### Clips the predicted probability (y_pred) to a small epsilon value (K.epsilon() ≈ 10^{-7}) 
        ### to prevent numerical instability when computing logarithms.
        y_pred = tf.clip_by_value(y_pred, K.epsilon(), 1 - K.epsilon())

        # Compute alpha dynamically based on class (element-wise multiplication)
         # IF y_true = 1 => alpha_t = y_true * alpha[1]
         # IF y_true = 0 => alpha_t = (1 - y_true) * alpha[0]
        alpha_t = y_true * alpha[1] + (1 - y_true) * alpha[0]

        # Calculate standard binary cross-entropy (BCE)
        cross_entropy = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)
        
        # Compute focal loss with class-specific alpha
            # tf.math.pow(1 - y_pred, gamma)
                # Applying a  power to the error (1-y_pred) it Down-weights well-classified examples 
                    # (small errors became smaller)
                    # Consider y_pred = P(is_fraud = 1)
                # Es:
                    # IF y_true = 1 and y_pred is close to 1 (1-y_pred) = small error became smaller
                    # IF y_true = 1 and y_pred is close to 0 (1-y_pred) = big error became bigger
            # alpha_t *
                # weights the loss result giving more importance to fraud class y_true = 1
        focal_loss = alpha_t * tf.math.pow(1 - y_pred, gamma) * cross_entropy
        
        return tf.reduce_mean(focal_loss)

    return loss_fn



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
import tensorflow as tf
from tensorflow.keras.metrics import AUC


def create_FFN(input_dim, num_blocks=3, initial_units=128, dropout_rate=0.3, activation='relu', 
               l1_reg=0.0, l2_reg=0.0, custom_focal_loss=False, alpha=0.25, gamma=2.0, 
               learning_rate=0.001, reduction_rate=0.5):
    """
    Create a dynamic Feedforward Neural Network with L1/L2 regularization, customizable layers, and other parameters.

    Parameters:
        input_dim (int): Input dimension for the first layer.
        num_blocks (int): Number of Dense-BatchNorm-Dropout blocks.
        initial_units (int): Number of units in the first Dense layer. Each subsequent block halves this number.
        dropout_rate (float): Dropout rate for Dropout layers.
        activation (str): Activation function for Dense layers (except the output layer).
        l1_reg (float): L1 regularization coefficient.
        l2_reg (float): L2 regularization coefficient.
        custom_focal_loss (bool): Whether to use a custom focal loss.
        alpha (float): Alpha parameter for focal loss.
        gamma (float): Gamma parameter for focal loss.
        learning_rate (float): Learning rate for the optimizer.

    Returns:
        model (tf.keras.Model): Compiled Keras model.
    """
    print("Model building...")

    # Hyperparameters dictionary for reference
    hyperparameters = {
        "num_blocks": num_blocks,
        "initial_units": initial_units,
        "dropout_rate": dropout_rate,
        "activation": activation,
        "l1_reg": l1_reg,
        "l2_reg": l2_reg,
        "custom_focal_loss": custom_focal_loss,
        "alpha": alpha,
        "gamma": gamma,
    }

    # Create a sequential model
    model = Sequential()

    # Add the first Dense layer with L1/L2 regularization
    units = initial_units
    model.add(Dense(units, input_dim=input_dim, activation=activation,
                    kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    # Add subsequent Dense-BatchNorm-Dropout blocks
    for layer_number in range(num_blocks - 1):
        units = max(1, int(units*reduction_rate))  # Ensure units do not go below 1
        model.add(Dense(units, activation=activation,
                        kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)))
        if layer_number < num_blocks - 2:
            model.add(BatchNormalization())
            model.add(Dropout(dropout_rate))

    # Add the output layer for binary classification
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    if custom_focal_loss:
        print("Warning: Using a custom focal loss")
        model.compile(
            optimizer=Adam(learning_rate=learning_rate),
            loss=focal_loss(alpha, gamma),
            metrics=['accuracy', 
                     AUC(curve='PR', name='pr_auc')  # Precision-Recall AUC  <-- #tf.keras.metrics.AUC(name='auc')
                    ]
        )
    else:
        model.compile(
            optimizer=Adam(learning_rate=learning_rate),
            loss='binary_crossentropy',
            metrics=['accuracy', 
                     AUC(curve='PR', name='pr_auc')  # Precision-Recall AUC  <-- #tf.keras.metrics.AUC(name='auc')
                    ]
        )

    # Print model summary
    print(model.summary())

    return model, hyperparameters


from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

from keras.models import Model
from keras.layers import Input, Dense, Dropout, BatchNormalization, LeakyReLU
from keras.optimizers import Adam

from keras.layers import Input, Dense, BatchNormalization, LeakyReLU, Dropout
from keras.models import Model
from keras.optimizers import Adam

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, LeakyReLU, Dropout
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam

def create_autoencoder(X_train, encoding_dim=6, reduction_rate=0.5, activation='LeakyReLU', dropout_rate=0.2, learning_rate=0.0001, l1=0.01, l2=0.01):
    input_dim = X_train.shape[1]

    input_layer = Input(shape=(input_dim,))
    encoded = input_layer

    # Encoder layers (store dimensions for symmetry)
    encoder_dims = []
    current_dim = input_dim
    while current_dim > encoding_dim:
        current_dim = max(encoding_dim, int(current_dim * reduction_rate))
        encoder_dims.append(current_dim)  # Store dimension
        encoded = Dense(current_dim, activation=None, kernel_regularizer=l1_l2(l1=l1, l2=l2))(encoded)
        encoded = BatchNormalization()(encoded)
        encoded = LeakyReLU(alpha=0.1)(encoded)
        encoded = Dropout(dropout_rate)(encoded)

    # Bottleneck (optional activation)
    # encoded = Activation('linear')(encoded)  # Or no activation at all

    # Decoder layers (reverse encoder dimensions)
    decoded = encoded
    for current_dim in reversed(encoder_dims):  # Reverse the dimensions
        decoded = Dense(current_dim, activation=None, kernel_regularizer=l1_l2(l1=l1, l2=l2))(decoded)
        decoded = BatchNormalization()(decoded)
        decoded = LeakyReLU(alpha=0.1)(decoded)

    decoded = Dense(input_dim, activation='linear')(decoded)  # Linear activation for output

    autoencoder = Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

    print(autoencoder.summary())
    return autoencoder, {  # Return hyperparameters as a dictionary
        "encoding_dim": encoding_dim,
        "dropout_rate": dropout_rate,
        "activation": activation,
        "learning_rate": learning_rate,
        "l1": l1,
        "l2": l2
    }




from tensorflow.keras.layers import Input, Dense, Lambda, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import numpy as np

def sampling(args):
    """Reparameterization trick: Sample latent space using mean and log variance."""
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=K.shape(z_mean), mean=0., stddev=1.0)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def create_variational_autoencoder(X_train, encoding_dim=6, dropout_rate=0.2, learning_rate=0.0001, l1=0.01, l2=0.01):
    input_dim = X_train.shape[1]
    
    # Input layer
    input_layer = Input(shape=(input_dim,))
    
    # Encoder
    encoded = Dense(64, activation=None, kernel_regularizer=l1_l2(l1=l1, l2=l2))(input_layer)  # Apply L1_L2 regularization
    encoded = BatchNormalization()(encoded)
    encoded = LeakyReLU(alpha=0.1)(encoded)
    encoded = Dense(32, activation=None, kernel_regularizer=l1_l2(l1=l1, l2=l2))(encoded)  # Apply L1_L2 regularization
    encoded = BatchNormalization()(encoded)
    encoded = LeakyReLU(alpha=0.1)(encoded)
    
    # Latent space
    z_mean = Dense(encoding_dim, name="z_mean")(encoded)
    z_log_var = Dense(encoding_dim, name="z_log_var")(encoded)
    z = Lambda(sampling, output_shape=(encoding_dim,), name="z")([z_mean, z_log_var])
    
    # Decoder
    decoded = Dense(32, activation=None, kernel_regularizer=l1_l2(l1=l1, l2=l2))(z)  # Apply L1_L2 regularization
    decoded = BatchNormalization()(decoded)
    decoded = LeakyReLU(alpha=0.1)(decoded)
    decoded = Dense(64, activation=None, kernel_regularizer=l1_l2(l1=l1, l2=l2))(decoded)  # Apply L1_L2 regularization
    decoded = BatchNormalization()(decoded)
    decoded = LeakyReLU(alpha=0.1)(decoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)
    
    # VAE model
    vae = Model(inputs=input_layer, outputs=decoded)
    
    # Define VAE loss
    reconstruction_loss = mse(input_layer, decoded)
    kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer=Adam(learning_rate=learning_rate))
    
    return vae, Model(inputs=input_layer, outputs=[z_mean, z_log_var])  # Return the VAE and encoder (for latent representations)



## Model Performance Evaluation

In [1]:
def plot_performance(history):
    fig, ax = plt.subplots(1, 3, figsize=(18, 5))  # Three subplots for Loss, Accuracy, and PR AUC
    fig.tight_layout(pad=4.0)

    # Extract metrics
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']

    if 'accuracy' in history.history:  # If accuracy is being tracked
        train_acc = history.history['accuracy']
        val_acc = history.history['val_accuracy']

        # Plot Loss
        ax[0].set_xlabel('Epochs')
        ax[0].set_ylabel('Loss')
        ax[0].set_title('Loss')
        ax[0].plot(train_loss, label='Training Loss')
        ax[0].plot(val_loss, label='Validation Loss')
        ax[0].legend()

        # Plot Accuracy
        ax[1].set_xlabel('Epochs')
        ax[1].set_ylabel('Accuracy')
        ax[1].set_title('Accuracy')
        ax[1].plot(train_acc, label='Training Accuracy')
        ax[1].plot(val_acc, label='Validation Accuracy')
        ax[1].legend()
    else:
        # Plot Loss only
        ax[0].set_xlabel('Epochs')
        ax[0].set_ylabel('Loss')
        ax[0].set_title('Loss')
        ax[0].plot(train_loss, label='Training Loss')
        ax[0].plot(val_loss, label='Validation Loss')
        ax[0].legend()
        ax[1].remove()  # Remove the unused subplot

    # Check if PR AUC exists
    if 'pr_auc' in history.history:
        train_pr_auc = history.history['pr_auc']
        val_pr_auc = history.history['val_pr_auc']

        # Plot Precision-Recall AUC
        ax[2].set_xlabel('Epochs')
        ax[2].set_ylabel('PR AUC')
        ax[2].set_title('Precision-Recall AUC')
        ax[2].plot(train_pr_auc, label='Training PR AUC')
        ax[2].plot(val_pr_auc, label='Validation PR AUC')
        ax[2].legend()
    else:
        ax[2].remove()  # Remove the unused subplot if no PR AUC

    plt.show()




def plot_confusion_matrix(cm):

    # Plotting confusion matrix using seaborn's heatmap
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', cbar=False, xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'])
    plt.title("Confusion Matrix")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
    
    
from sklearn.metrics import average_precision_score    

def compute_main_scores(y_test, y_pred, y_pred_prob):
    accuracy = accuracy_score(y_test, y_pred)
    # Compute AUC
    pr_auc = average_precision_score(y_test, y_pred_prob)
    # Print the results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision-Recall AUC Score: {pr_auc:.4f}")

    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    plot_confusion_matrix(cm)
    
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)


    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, digits=4))
    
    return accuracy, pr_auc, recall, f1, precision

    
def evaluation_on_test(model, X_test, y_test, best_threshold=0.5):

    # Predictions and classification report
    y_pred_prob = model.predict(X_test).flatten()
    y_pred = (y_pred_prob >= best_threshold).astype(int)
    
    compute_main_scores(y_test, y_pred, y_pred_prob)
    
    
from sklearn.metrics import precision_recall_curve, f1_score, roc_auc_score, confusion_matrix, precision_score, recall_score

def threshold_metric_evaluation(model, X_val, y_val):
    
    # Get probabilities on validation data
    y_pred_prob = model.predict(X_val).flatten()
    
    # Define thresholds
    thresholds = np.arange(0.0, 1.01, 0.01)
    
    # Initialize metrics storage
    precision_scores = []
    recall_scores = []
    f1_scores = []
    
    # Evaluate metrics for each threshold
    for thresh in thresholds:
        y_pred_thresh = (y_pred_prob >= thresh).astype(int)
        precision = precision_score(y_val, y_pred_thresh, zero_division=0)
        recall = recall_score(y_val, y_pred_thresh)
        f1 = f1_score(y_val, y_pred_thresh)

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
    
    # Filter thresholds that satisfy recall >= 0.7
    valid_indices = [i for i, recall in enumerate(recall_scores) if recall >= 0.7]
    
    if not valid_indices:
        print("No thresholds satisfy the recall constraint of 0.7.")
        return

    # Among valid thresholds, find the one with the best F1-Score
    valid_f1_scores = [f1_scores[i] for i in valid_indices]
    best_threshold_idx = valid_indices[np.argmax(valid_f1_scores)]
    best_threshold = thresholds[best_threshold_idx]
    
    print(f"Best Threshold (Recall >= 0.7 & Max F1-Score): {best_threshold:.2f}")
    print(f"Precision: {precision_scores[best_threshold_idx]:.2f}, Recall: {recall_scores[best_threshold_idx]:.2f}, F1-Score: {f1_scores[best_threshold_idx]:.2f}")
    
    # Plot the metrics
    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, precision_scores, label="Precision", marker='o')
    plt.plot(thresholds, recall_scores, label="Recall", marker='o')
    plt.plot(thresholds, f1_scores, label="F1-Score", marker='o')
    plt.axvline(best_threshold, color='r', linestyle='--', label=f"Best Threshold ({best_threshold:.2f})")
    plt.xlabel("Decision Threshold")
    plt.ylabel("Metric Value")
    plt.title("Precision, Recall, and F1-Score vs. Decision Threshold")
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return best_threshold

def get_experiments_log():
    csv_path = "experiment_log.csv"
    df = pd.read_csv(csv_path)
    
    return df


import os
from datetime import datetime

def update_experiment_log(csv_path, experiment_data):
    """
    Update the experiment log with new experiment data and save it to a CSV file.

    Parameters:
        csv_path (str): The file path to the CSV file.
        experiment_data (dict): A dictionary containing the experiment details:
            - timestamp (str): The experiment timestamp.
            - model_type (str): The type of model used.
            - note (str): A personal note about the experiment.
            - hyperparameters (dict): A dictionary of hyperparameters used.
            - excluded_features (list): The set of feature names removed for the experiment.
            - best_threshold (float): The best threshold used for classification.
            - val_precision (float): The validation precision metric.
            - val_recall (float): The validation recall metric.
            - val_f1_score (float): The validation F1-score.
            - val_accuracy (float): The validation accuracy metric.
            - val_auc (float): The validation AUC metric.
            - val_aic (float): The validation AIC value.
            - val_smd (float): The validation SMD value.
            - test_precision (float): The test precision metric.
            - test_recall (float): The test recall metric.
            - test_f1_score (float): The test F1-score.
            - test_accuracy (float): The test accuracy metric.
            - test_auc (float): The test AUC metric.
            - test_aic (float): The test AIC value.
            - test_smd (float): The test SMD value.
    """
    # Convert the dictionary of hyperparameters to a JSON-like string
    experiment_data["hyperparameters"] = str(experiment_data["hyperparameters"])

    # Convert the list of features to a comma-separated string
    experiment_data["excluded_features"] = "- ".join(experiment_data["excluded_features"])

    # Check if the CSV file exists
    if os.path.exists(csv_path):
        # Load the existing DataFrame
        df = pd.read_csv(csv_path)
    else:
        # Create a new DataFrame
        columns = [
            "timestamp", "model_type", "tuning_on", "features_note", "dataset_size", "note", "hyperparameters", 
            "excluded_features", "best_threshold",
            "val_precision", "val_recall", "val_f1", "val_accuracy", "val_auc", "val_aic", "val_smd",
            "test_precision", "test_recall", "test_f1", "test_accuracy", "test_auc", "test_aic", "test_smd",
    
        ]
        df = pd.DataFrame(columns=columns)

    # Append the new data to the DataFrame
    df = pd.concat([df, pd.DataFrame([experiment_data])], ignore_index=True)

    # Save the updated DataFrame to the CSV file
    df.to_csv(csv_path, index=False)
    
    
def save_experiment_data(experiment_data, csv_path):
    """
    Save experiment data to a CSV file.

    Parameters:
        experiment_data (dict): A dictionary containing the experiment data.
        file_name (str): The name of the CSV file to save the data.
    """
    # Define the header based on the keys of the dictionary
    header = list(experiment_data.keys())

    # Check if the file exists
    file_exists = os.path.isfile(csv_path)

    # Open the file in append mode
    with open(csv_path, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header)

        # Write the header only if the file does not exist
        if not file_exists:
            writer.writeheader()

        # Write the experiment data
        writer.writerow(experiment_data)

    print(f"Experiment data saved to {csv_path}")

    

def compare_test_performance(csv):
    # Load the dataset
    performance_df = pd.read_csv(csv)

    import seaborn as sns
    import matplotlib.pyplot as plt

    # Create a single figure with two subplots
    fig, axes = plt.subplots(1, 5, figsize=(12, 4), sharey=True)
    
    # Plot Test F1 vs Tuning On
    sns.barplot(ax=axes[0], x='test_accuracy', y='model_type', data=performance_df, palette='viridis')
    axes[0].set_xlabel('Test Accuracies')
    axes[0].set_ylabel('')  # No y-label for the second plot
    axes[0].set_title('Test Accuracies')
    axes[0].grid(alpha=0.25)

    # Plot Test Recall vs Tuning On
    sns.barplot(ax=axes[1], x='test_recall', y='model_type', data=performance_df, palette='viridis')
    axes[1].set_xlabel('Test Recall')
    axes[1].set_ylabel('')
    axes[1].set_title('Test Recall')
    axes[1].grid(alpha=0.25)

    # Plot Test F1 vs Tuning On
    sns.barplot(ax=axes[2], x='test_f1', y='model_type', data=performance_df, palette='viridis')
    axes[2].set_xlabel('Test F1')
    axes[2].set_ylabel('')  # No y-label for the second plot
    axes[2].set_title('Test F1')
    axes[2].grid(alpha=0.25)
    
    # Plot Test F1 vs Tuning On
    sns.barplot(ax=axes[3], x='test_auc', y='model_type', data=performance_df, palette='viridis')
    axes[3].set_xlabel('Test PR AUC')
    axes[3].set_ylabel('')  # No y-label for the second plot
    axes[3].set_title('Test PR AUC')
    axes[3].grid(alpha=0.25)
    
    # test_aic
    sns.barplot(ax=axes[4], x='test_aic', y='model_type', data=performance_df, palette='viridis')
    axes[4].set_xlabel('Test AIC')
    axes[4].set_ylabel('')  # No y-label for the second plot
    axes[4].set_title('Test AIC')
    axes[4].grid(alpha=0.25)
    


    # Adjust layout for better spacing
    plt.tight_layout()
    plt.show()


    
import csv
import os

# Define a function to save training information
def save_validation_info(validation_performance, csv_file):
    # Check if the file exists
    file_exists = os.path.exists(csv_file)
    
    # Open the file in append mode
    with open(csv_file, mode="a", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=validation_performance.keys())
        
        # Write the header only if the file is new
        if not file_exists:
            writer.writeheader()
        
        # Write the current training information
        writer.writerow(validation_performance)






## Fairness Evaluation Utils

In [14]:
def adverse_impacat_ratio(df, marker, val, control_val, target, target_val):
    
    control_pop = df[df[marker]==control_val]
    control_number = control_pop[marker].count()
    control_pos_target = control_pop[df[target]==target_val][marker].count()
    
    control_correct_target = control_pop[df[target]==df['label']][marker].count()
    
    # ratios
    control_pos_rate = control_pos_target / control_number
    control_accuracy = control_correct_target / control_number
    
    
    marker_pop = df[df[marker]==val]
    pop_number = marker_pop[marker].count()
    pop_pos_target = marker_pop[df[target]==target_val][marker].count()

    pop_correct_target = marker_pop[df[target]==df['label']][marker].count()

    #ratios
    pop_pos_rate = pop_pos_target / pop_number
    pop_accuracy = pop_correct_target / pop_number

    AIR_outcome = pop_pos_rate / control_pos_rate
    # Unfair in accuracy metric
    AIR_accuracy = pop_accuracy / control_accuracy

        
    return AIR_outcome, AIR_accuracy
    
    
    
def standardized_mean_difference(df, marker, val, control_val, target):
    # small 0.2, 0.5, and 0.8 large

    control_values = df[df[marker] == control_val][target].values
    control_mean = np.mean(control_values)
    control_std = np.std(control_values)


    pop_values = df[df[marker] == val][target].values
    pop_mean = np.mean(pop_values)
    pop_std = np.std(pop_values)

    smd = (pop_mean - control_mean) / np.sqrt((pop_std**2 + control_std**2) / 2)

    return smd



def get_final_result_df(labels, predictions, X_test):
    
    print(labels.shape)
    print(predictions.shape)
    print(X_test.shape)
    
    results_df = pd.DataFrame({'label': labels, 'prediction': predictions})

    # Concatenate the original test data with the results DataFrame
    final_results_df = pd.concat([X_test, results_df], axis=1)

    return final_results_df



####### FAIRNESS METRIC EVALUATION:
# es: demographich_marker="gender", pop=0, control_pop= 1, pos_outcome=0
def fairness_evaluation(model, X_val, y_val, y_val_pred_classes, demographich_marker, pop, control_pop, pos_outcome):

    conc_result_df = get_final_result_df(y_val, y_val_pred_classes.ravel(), X_val)                        
    smd = standardized_mean_difference(conc_result_df, demographich_marker, 0, 1, 'prediction')
    air_outcome, air_out_accuracy = adverse_impacat_ratio(conc_result_df, demographich_marker, 0, 1, 'prediction', 0)
    
    print(f"Standardized mean difference of non-fraud {demographich_marker}: ", smd)
    print(f"Adverse Impact Ratio of non-fraud for {demographich_marker}: ", air_outcome)
    print(f"Adverse Impact Ratio of accuracy for {demographich_marker}: ", air_out_accuracy)
    
    return conc_result_df, smd, air_outcome, air_out_accuracy



def create_biased_df(df,  biased_fraction):

    biased_df = df.copy()

    # make it deterministic
    np.random.seed(42)


    female_shape = biased_df[biased_df['sex'] == 0].shape[0]
   
    biased_indices = np.random.choice(biased_df[biased_df['sex'] == 0][biased_df['credit_risk'] != 0].index, 
                                      size=int(biased_fraction * female_shape), replace=False)
    biased_df.loc[biased_indices, 'credit_risk'] = 0  

    air_score = adverse_impact_ratio_for_df(biased_df, 'sex', 0, 'credit_risk')
    print(f"Adverse Impact Ratio (AIR): {air_score}")
    
    sme_score = standardized_mean_difference_for_df(biased_df, 'sex', 0, 1, 'credit_risk')
    print(f"Standardized Mean Difference {sme_score}")
    
    return biased_df
